In [1]:
pip install googletrans==4.0.0-rc1

In [2]:
!pip install pytube

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import json

api_key = ''
file_path = '/content/API.json'

try:
    with open(file_path, 'r') as file_to_read:
        json_data = json.load(file_to_read)
        api_key = json_data["API_KEY"]

except FileNotFoundError:
    print("JSON file not found at the specified path:", file_path)

except KeyError:
    print("API_KEY key not found in the JSON data.")

except json.JSONDecodeError as e:
    print("Error decoding JSON data:", str(e))

**Install python library for YouTube**

**Importing YouTube library from pytube. The user gets 2 options such that it can upload a youtube video or else it can upload video from the drive to get audio from the video.**

In [5]:
print('For Youtube video enter 1 and for drive video enter 0')
n=int(input('ENTER = '))
if n==1:
  link=input('ENTER THE LINK : ')
  from pytube import YouTube
  video=YouTube(link)
  yt=video.streams.get_audio_only()
  k=yt.download()
elif n==0:
  link=input('ENTER THE LINK : ')
  k=link
else:
  print('Sorry wrong input.')

For Youtube video enter 1 and for drive video enter 0
ENTER = 1
ENTER THE LINK : https://www.youtube.com/watch?v=EpZb7mnMHwQ&pp=ygUSc3RvcmllcyBpbiBlbmdsaXNo


**Uploading the audio data retrieved from the YouTube video and feeding into the model to get the transcript.**

In [6]:
import sys
import time
import requests

file1 = k
def read_file(file1, size = 5242880):
  with open(file1, 'rb') as file:
    while True:
      data = file.read(size)
      if not data:
        break
      yield data
headers = {'authorization' : api_key}
response = requests.post('https://api.assemblyai.com/v2/upload', headers = headers, data = read_file(file1))
audio_url = response.json()['upload_url']
print(audio_url)

https://cdn.assemblyai.com/upload/e18202b5-dbe9-4343-94ea-a792b90cc45f


**This step performs authorization and perform transcript input response**

In [7]:
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {"audio_url": audio_url}

headers = {"authorization" : api_key, "content_type": "application/json"}

transcript_input_response = requests.post(endpoint, json = json, headers = headers)

temp1 = transcript_input_response.json()
temp1

{'id': 'e2b7c25b-9e4a-4a34-92b1-1c7139a5fbdf',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/e18202b5-dbe9-4343-94ea-a792b90cc45f',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': False,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_categ

In [8]:
transcript_id = transcript_input_response.json()["id"]
transcript_id

'e2b7c25b-9e4a-4a34-92b1-1c7139a5fbdf'

**The transcript output status is reflected. The status reflected can be processing or completed.**

In [9]:
endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers = {"authorization": api_key, }

condition='not started'
while condition!='completed':
    transcript_output_response = requests.get(endpoint, headers = headers)
    condition = transcript_output_response.json()["status"]

In [10]:
temp2 = transcript_output_response.json()
temp2

{'id': 'e2b7c25b-9e4a-4a34-92b1-1c7139a5fbdf',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/e18202b5-dbe9-4343-94ea-a792b90cc45f',
 'text': "Twin Twinkies. Real education. Tinku and Rinky were twin siblings. They both studied in the same class in the same school. Both were excellent in studies. But often Tinku scored better than his sister. Dad laid down a condition when both of them insisted on having a bicycle. My budget allows me to purchase only one bicycle. So whoever will score better between you two will get the bicycle. After hearing father's words, both the kids started studying. But Tinku was very smart. He knew how to get good marks. He used to study only that portion which was supposed to come in the exams. Whereas Rinky was fond of studies. She used to study the entire textbook. Keep in mind, I will get the bicycle. Whatever you are study

In [11]:
import datetime
for segment in temp2['words']:
    start_time = segment['start']
    end_time = segment['end']

    time_format1 = datetime.timedelta(milliseconds=start_time)
    time_string1 = str(time_format1)
    time_string1 = time_string1[:-7] + ',' + time_string1[-6:]
    segment['start'] = time_string1.zfill(15)[:-3]

    time_format2 = datetime.timedelta(milliseconds=end_time)
    time_string2 = str(time_format2)
    time_string2 = time_string2[:-7] + ',' + time_string2[-6:]
    segment['end'] = time_string2.zfill(15)[:-3]
temp2['words']

[{'start': '00:00:00,280',
  'end': '00:00:00,456',
  'text': 'Twin',
  'confidence': 0.86436,
  'speaker': None},
 {'start': '00:00:00,456',
  'end': '00:00:01,388',
  'text': 'Twinkies.',
  'confidence': 0.06005,
  'speaker': None},
 {'start': '00:00:05,694',
  'end': '00:00:06,246',
  'text': 'Real',
  'confidence': 0.99,
  'speaker': None},
 {'start': '00:00:06,350',
  'end': '00:00:06,994',
  'text': 'education.',
  'confidence': 0.99382,
  'speaker': None},
 {'start': '00:00:07,734',
  'end': '00:00:08,334',
  'text': 'Tinku',
  'confidence': 0.91224,
  'speaker': None},
 {'start': '00:00:08,374',
  'end': '00:00:08,526',
  'text': 'and',
  'confidence': 1.0,
  'speaker': None},
 {'start': '00:00:08,550',
  'end': '00:00:09,182',
  'text': 'Rinky',
  'confidence': 0.49281,
  'speaker': None},
 {'start': '00:00:09,278',
  'end': '00:00:09,558',
  'text': 'were',
  'confidence': 0.99719,
  'speaker': None},
 {'start': '00:00:09,606',
  'end': '00:00:09,878',
  'text': 'twin',
  'co

In [12]:
transcript_with_timeframes = temp2['words']
output_data = []  # List to store the output sentences

sentence = ''
flag = True

for segment in transcript_with_timeframes:
    start_time = segment['start']
    end_time = segment['end']
    text = segment['text']

    if flag:
        output_data.append(start_time+" --> ")

    if text[-1] != '.' and text[-1] != '?':
        sentence += text + " "
        flag = False

    else:
        sentence += text
        output_data.append(end_time+"\n"+sentence+"\n")  # Store the sentence in the output_data list
        #output_data.append(end_time)
        sentence = ''
        flag = True

#Convert the list into a single string with line breaks

data = ''.join(output_data)
print(data)

00:00:00,280 --> 00:00:01,388
Twin Twinkies.
00:00:05,694 --> 00:00:06,994
Real education.
00:00:07,734 --> 00:00:10,814
Tinku and Rinky were twin siblings.
00:00:10,974 --> 00:00:14,934
They both studied in the same class in the same school.
00:00:15,094 --> 00:00:17,182
Both were excellent in studies.
00:00:17,318 --> 00:00:20,354
But often Tinku scored better than his sister.
00:00:26,254 --> 00:00:31,914
Dad laid down a condition when both of them insisted on having a bicycle.
00:00:33,414 --> 00:00:36,542
My budget allows me to purchase only one bicycle.
00:00:36,638 --> 00:00:40,914
So whoever will score better between you two will get the bicycle.
00:00:41,534 --> 00:00:45,674
After hearing father's words, both the kids started studying.
00:00:45,974 --> 00:00:48,474
But Tinku was very smart.
00:00:48,894 --> 00:00:51,194
He knew how to get good marks.
00:00:51,664 --> 00:00:56,684
He used to study only that portion which was supposed to come in the exams.
00:00:58,264 --> 00:01

**Gives the output of the transcript.**

In [13]:
data1 = transcript_output_response.json()["text"]
data1

"Twin Twinkies. Real education. Tinku and Rinky were twin siblings. They both studied in the same class in the same school. Both were excellent in studies. But often Tinku scored better than his sister. Dad laid down a condition when both of them insisted on having a bicycle. My budget allows me to purchase only one bicycle. So whoever will score better between you two will get the bicycle. After hearing father's words, both the kids started studying. But Tinku was very smart. He knew how to get good marks. He used to study only that portion which was supposed to come in the exams. Whereas Rinky was fond of studies. She used to study the entire textbook. Keep in mind, I will get the bicycle. Whatever you are studying is useless. This wont come in exams. How do you know? See this. Tushan sir has given me question papers of last three years. Every time questions are repeated from these only. Okay. But I dont want to take any shortcut. Whatever will be the question, it will be asked from 

**Enables user to download the transcript for the corresponding video.**

In [14]:
language_codes = {
    'english': 'en',
    'spanish': 'es',
    'chinese': 'zh',
    'hindi': 'hi',
    'arabic': 'ar',
    'portuguese': 'pt',
    'bengali': 'bn',
    'russian': 'ru',
    'japanese': 'ja',
    'punjabi': 'pa',
    'german': 'de',
    'french': 'fr',
    'italian': 'it',
    'korean': 'ko',
    'turkish': 'tr',
    'telugu': 'te',
    'marathi': 'mr',
    'tamil': 'ta',
    'vietnamese': 'vi',
    'urdu': 'ur'
}

In [15]:
from googletrans import Translator

def translate_english_to_other(text,ln):
    if text is None or text.strip() == '':
        return ''  # Return an empty string if the input is None or empty
    translator = Translator()
    translation = translator.translate(text, src='en', dest=ln)
    other_translation = translation.text
    return other_translation

In [16]:
lang=input("Enter language to be translated to: ").lower()

Enter language to be translated to: english


In [17]:
if lang not in language_codes.keys():
  print("Enter valid language")

In [18]:
langcode = language_codes[lang.lower()]
langcode

'en'

In [19]:
english_text = data[:50]
other_text = translate_english_to_other(english_text,langcode)

print("English: ", english_text)
print("Translated: ", other_text)

English:  00:00:00,280 --> 00:00:01,388
Twin Twinkies.
00:00
Translated:  00:00:00,280 --> 00:00:01,388
Twin Twinkies.
00:00


In [20]:
sentences=[]
tempsent=''
for i in data:
    if i!="\n":
        tempsent+=i
    else:
        sentences.append(tempsent)
        tempsent=''

In [21]:
f = open('subtitle.srt', 'w')

i=j=0
for sentence in sentences:
    i+=1
    if i%2==1:
        j+=1
        f.write(str(j))
        f.write("\n")
        f.write(sentence)
        f.write("\n")
    else:
        other_text = translate_english_to_other(sentence,langcode)
        f.write(other_text)
        f.write("\n\n")
f.close()

In [22]:
from google.colab import files

print('If you want to download captions file enter 0 else enter 1 :- ')
m=int(input(''))
if m==0:
  transcript_file_path = '/content/subtitle.srt'
  try:
    files.download(transcript_file_path)
    print("Thank you! Downloaded successfully")
  except OSError as e:
      print(f"An error occurred while opening the file: {e}")
elif m==1:
  print('You do not want to download.Thank You...')
else:
  print('Sorry Wrong input.')

If you want to download enter 0 else enter 1 :- 
1
You do not want to download.Thank You...


**Third Part of the project is to summarize the content of the SRT file**

In [24]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset
import torch
import base64
import random
from transformers import pipeline

In [30]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [31]:
pip install accelerate -U

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

# Downloading NLTK resources
nltk.download('punkt')

In [11]:
def train_model(train_inputs, train_targets, val_inputs, val_targets, checkpoint="MBZUAI/LaMini-Flan-T5-248M"):
    tokenizer = T5Tokenizer.from_pretrained(checkpoint)
    model = T5ForConditionalGeneration.from_pretrained(checkpoint)

    # Tokenizing inputs and targets
    train_encodings = tokenizer(train_inputs, truncation=True, padding=True)
    train_labels = tokenizer(train_targets, truncation=True, padding=True)
    val_encodings = tokenizer(val_inputs, truncation=True, padding=True)
    val_labels = tokenizer(val_targets, truncation=True, padding=True)

    # Preparing datasets
    train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels["input_ids"]})
    val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels["input_ids"]})

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./t5_summaries",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=50
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    return model

def parse_srt_file(file_path):
    subtitles = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        subtitle = ''
        for line in lines:
            line = line.strip()
            if line.isdigit():
                if subtitle:
                    subtitles.append(subtitle)
                    subtitle = ''
            elif line:
                if "-->" not in line:
                    subtitle += line + ' '
    return subtitles


# File preprocessing
def srt_preprocessing(file_path):
    subtitles = parse_srt_file(file_path)
    text = ' '.join(subtitles)
    # Split the text into training and testing datasets
    sentences = text.split('.')
    random.shuffle(sentences)
    split_index = int(len(sentences) * 0.8)  # 80% for training, 20% for testing
    train_text = sentences[:split_index]
    test_text = sentences[split_index:]
    return train_text, test_text

# LLM pipeline
def llm_pipeline_srt(train_text, test_text):
    model = train_model(train_text, train_text, test_text, test_text)
    tokenizer = T5Tokenizer.from_pretrained(model.name_or_path)
    pipe_sum = pipeline(
        'summarization',
        model=model,
        tokenizer=tokenizer,
        max_length=100,
        min_length=1)
    result = pipe_sum(test_text)
    summary = result[0]['summary_text']
    return summary


file_path = "subtitle.srt"
train_text, test_text = srt_preprocessing(file_path)
summary = llm_pipeline_srt(train_text, test_text)
print("Summarization Complete")
print("Summary:")
print(summary)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1542 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 2000, but your input_length is only 1542. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=771)


Summarization Complete
Summary:
Twin Twinkies, Tinku and Rinky, are twin siblings who studied in the same school. Tinku was excellent in studies but often scored better than his sister. Rinky was fond of studies and solved all the questions correctly. Rinku was awarded a bicycle as a prize along with other gifts. The family went out for dinner to celebrate the results of both kids. Sonu and Vani's exams are coming soon, and Sonu is worried about how he will complete his studies if he doesn't study on time. The teacher scolds Sonu for not submitting his project, but Sonu learns from his sister how to utilize time properly. The weather turns into a storm and rains are expected to happen, and the children are worried about what they will write in their exams tomorrow. Both kids went happily to appear in exams.


In [13]:
f = open('summarized.txt', 'w')
f.write(summary)
f.close()

In [16]:
from google.colab import files

print('If you want to download summarized text file enter 0 else enter 1 :- ')
m=int(input(''))
if m==0:
  transcript_file_path = '/content/summarized.txt'
  try:
    files.download(transcript_file_path)
    print("Thank you! Downloaded successfully")
  except OSError as e:
      print(f"An error occurred while opening the file: {e}")
elif m==1:
  print('You do not want to download.Thank You...')
else:
  print('Sorry Wrong input.')

If you want to download summarized text file enter 0 else enter 1 :- 
0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Thank you! Downloaded successfully
